In [ ]:
import qulacs
from qulacs import QuantumState, QuantumCircuit
from qulacs.state import partial_trace
from qulacs.gate import DenseMatrix, CPTP
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
%matplotlib inline
#import physics-tenpy
#%load_ext wurlitzer

In [ ]:
# utils
def add_measurement(circuit, i, p):
    gate_0 = DenseMatrix(i, [[np.sqrt(1-p), 0], [0, np.sqrt(1-p)]])
    gate_1 = DenseMatrix(i, [[np.sqrt(p), 0], [0, 0]])
    gate_2 = DenseMatrix(i, [[0, 0], [0, np.sqrt(p)]])
    gate_list = [gate_0, gate_1, gate_2]
    gate = CPTP(gate_list)
    circuit.add_gate(gate)
def add_LRC(circuit, depth, l, r):
    #[l, r)
    for d in range(depth):
        for i in range(l + d % 2, r - 1, 2):
            circuit.add_random_unitary_gate([i, i + 1])
    return circuit
def add_RC(circuit, depth, l, r):
    #[l, r)
    for d in range(depth//2):
        for a in range(l, r):
            b = np.random.randint(l, r)
            while b == a:
                b = np.random.randint(l, r)
            circuit.add_random_unitary_gate([a, b])
    return circuit
def add_RLC_and_measurement(circuit, depth, l, r, p):
    #[l, r)
    circuit.add_random_unitary_gate(range(l, r))
    for d in range(depth):
        for i in range(l + d % 2, r - 1, 2):
            circuit.add_random_unitary_gate([i, i + 1])
        for i in range(l, r):
            add_measurement(circuit, i, p)
    circuit.add_random_unitary_gate(range(l, r))
    return circuit
def draw_graph(l, r, ave, std):
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.errorbar(range(l, r), ave, yerr=std, capsize=5)
    fig.show()

In [ ]:
## blackholes

# [0, k): Charlie
# [k, 2k): Alice
# [2k, n+k): Black hole -> [k, n+k)
def young_blackhole(n, k):
    circuit = QuantumCircuit(n+k)
    for i in range(k):
        circuit.add_H_gate(i)
        circuit.add_CNOT_gate(i, i + k)
    circuit.add_random_unitary_gate(list(range(k, n+k)))
    return circuit
def LRC_young_blackhole(n, k, depth):
    circuit = QuantumCircuit(n+k)
    for i in range(k):
        circuit.add_H_gate(i)
        circuit.add_CNOT_gate(i, i + 1)
    add_LRC(circuit, depth, k, n+k)
    return circuit  
def RC_young_blackhole(n, k, depth):
    circuit = QuantumCircuit(n+k)
    for i in range(k):
        circuit.add_H_gate(i)
        circuit.add_CNOT_gate(i, i + 1)
    add_RC(circuit, depth, k, n+k)
    return circuit 
def LRC_and_measurement_young_blackhole(n, k, depth, p):
    circuit = QuantumCircuit(n+k)
    for i in range(k):
        circuit.add_H_gate(i)
        circuit.add_CNOT_gate(i, i + 1)
    add_RLC_and_measurement(circuit, depth, k, n+k, p)
    return circuit 
# additional
def old_blackhole(n, k):
    circuit = QuantumCircuit(2 * n)
    for i in range(k):
        circuit.add_H_gate(i)
        circuit.add_gate(CNOT(i, i + k))
    for i in range(2 * k, n + k):
        circuit.add_H_gate(i)
        circuit.add_CNOT_gate(i, i + n - k)
    circuit.add_random_unitary_gate(list(range(k, n+k)))
    return circuit    

In [ ]:
# simulator for young black hole
def simulate_young(n, k, l_max, unitary_type="haar", depth=-1, iter_num=10):
    data = np.zeros((l_max + 1, iter_num))
    for l in range(l_max + 1):
        mat_size = pow(2, n+k-l)
        for i in range(iter_num):
            if i%10 == 0:
                print(f"l={l}:i-th iteration")
            state = qulacs.QuantumState(n + k)
            if unitary_type == "haar":
                circuit = young_blackhole(n, k)
            elif unitary_type == "RLC":
                circuit = LRC_young_blackhole(n, k, depth)# RCに書き換える
            elif unitary_type == "RC":
                pass
            elif unitary_type == "measure":
                circuit = LRC_and_measurement_young_blackhole(n, k, depth, 0.2)
            circuit.update_quantum_state(state)
            # trace out [n+k-l, n+k)
            trace = qulacs.state.partial_trace(state, list(range(n + k - l, n + k)))
            data[l][i] = npl.norm(trace.get_matrix() - np.identity(mat_size)/mat_size, 'nuc')
    return data

In [ ]:
data = simulate_young(9, 1, 9, iter_num=3)
np.savetxt("data/young.csv", data, delimiter=',')
#data = np.loadtxt('data/young.csv', delimiter=',')

In [ ]:
depth_list = [0, 1, 5, 10, 15, 20]
for d in depth_list:
    data = simulate_young(9, 1, 9, iter_num=2, unitary_type="LRC", depth=d)
    np.savetxt(f"data/LRC-{d}.csv", data, delimiter=',')

In [11]:
depth_list = [0, 1, 5, 10, 15, 20]
for d in depth_list:
    data = simulate_young(9, 1, 9, iter_num=10, unitary_type="measure", depth=d)
    np.savetxt(f"data/LRC_measurement-{d}.csv", data, delimiter=',')

In [ ]:
depth_list = [0, 1, 5, 10, 15, 20]
fig = plt.figure()
ax = fig.add_subplot()
for d in depth_list:
    data = np.loadtxt(f"data/LRC_measurement-{d}.csv", delimiter=',')
    ax.plot(range(10), np.average(data, axis=1), label=f"{d}")
ax.legend()
fig.show()


In [ ]:
depth_list = [0, 1, 5, 10, 15, 20]
fig = plt.figure()
ax = fig.add_subplot()
for d in depth_list:
    data = np.loadtxt(f"data/LRC_measurement-{d}.csv", delimiter=',')
    ax.plot(range(10), np.average(data, axis=1), label=f"{d}")
ax.legend()
fig.show()


In [ ]:
# old black hole (additional)
k = 1
n = 4
ave = []
var = []
for l in range(k, n+k):
    mat_size = pow(2, n+k-l)
    distance = []
    for i in range(10):
        dm = qulacs.DensityMatrix(n * 2)
        circuit = old_blackhole(n, k)
        circuit.update_quantum_state(dm)
        dm2 = qulacs.state.partial_trace(dm, list(range(n + k - l, n * 2)))
        mat_size = pow(2, n + k - l)
        distance.append(dist(dm2.get_matrix(), np.identity(mat_size)/mat_size))
    distance = np.array(distance)
    ave.append(np.average(distance))
    var.append(np.var(distance))